In [ ]:
import scipy.linalg.lapack as lapak

# Recursos quânticos
## Outras funções necessárias

In [3]:
def rho_basis_n(th, ph, rho, typ): # representation of rho in a given basis
    ket_n0, bra_n0, ket_n1, bra_n1 = n_basis(th, ph, typ)
    if typ == 's':
        rhon = Matrix([[bra_n0*rho*ket_n0, bra_n0*rho*ket_n1],[bra_n1*rho*ket_n0, bra_n1*rho*ket_n1]])
    elif typ == 'n':
        rho00 = bra_n0.dot(rho.dot(ket_n0)); rho01 = bra_n0.dot(rho.dot(ket_n1))
        rho10 = bra_n1.dot(rho.dot(ket_n0)); rho11 = bra_n1.dot(rho.dot(ket_n1))
        rhon = np.array([[rho00.item(), rho01.item()],[rho10.item(), rho11.item()]])
    return rhon

In [1]:
def Pauli(j):
    if j == 0:
        return np.array([[1,0],[0,1]])
    elif j == 1:
        return np.array([[0,1],[1,0]])
    elif j == 2:
        return np.array([[0,-1j],[1j,0]])
    elif j == 3:
        return np.array([[1,0],[0,-1]])

In [ ]:
ev = lapak.zheevd(Pauli(1)); ev, ev[0], ev[0][0], ev[1][0], ev[1][0][0]

In [ ]:
# Outside this function, initialize: evals = zeros(d,1)
def eVals(A):
    d = A.shape[0]; evals = zeros(d,1)
    eig = A.eigenvects()
    ne = 0
    j = 0
    lk = 0
    while ne < d:
        mult = eig[j][1]
        ne += mult
        nk = lk + mult
        for k in range(lk,nk):
            evals[k] = eig[j][0]
        lk = nk
        j += 1
    return evals

## Entropia

### Shannon entropy
For a probability vector $\vec{p}=[p_{1},\cdots,p_{d}]^{T}$,
\begin{equation}
H(\vec{p}) = -\sum_{j=1}^{d}p_{j}\log_{2}(p_{j})
\end{equation}
If $p_{j}=1$ for some $j$ then $H(\{1,0,\cdots,0\})=0$. If $p_{j}=1/d\ \forall\ j$ then $H(\{1/d,\cdots,1/d\}) = -\sum_{j=1}^{d}\frac{1}{d}\log_{2}\frac{1}{d}=d\frac{1}{d}\log_{2}(d) = \log_{2}(d).$ So, if we define $\bar{H}(\vec{p}) = \frac{H(\vec{p})}{\log_{2}(d)} \in [0,1].$

In [1]:
def shannon_num(pv):
    d = pv.shape[0]; SE = 0.0; j = -1
    while (j < d-1):
        j = j + 1
        if pv[j] > 10**-15 and pv[j] < (1.0-10**-15):
            SE -= pv[j]*math.log(pv[j], 2)
    return SE

In [ ]:
def shannon(pv): # for symbolics
    d = pv.shape[0]; H = 0
    for j in range(0,d):
        #if pv[j] > 10**-15 and pv[j] < (1-10**-15):
        H -= pv[j]*log(pv[j],2)
    return H

### von Neumann entropy
\begin{equation}
S_{vn}(\rho) = -Tr(\rho\log_{2}(\rho))
\end{equation}
Here $S_{vn}(|\psi\rangle)=0$ and $S_{vn}(\mathbb{I}/d)=\log_{2}(d)$. So $\bar{S}_{vn}(\rho)=\frac{S_{vn}(\rho)}{\log_{2}(d)}\in[0,1]$.

In [2]:
def von_neumann_num(rho):
    d = rho.shape[0]; d = lapak.zheevd(rho)
    return shannon_num(d[0])

In [ ]:
def von_neumann_num_2(rho):
    U, s, _ = np.linalg.svd(rho)
    return -np.sum(s*np.log2(s))

In [ ]:
def von_neumann_num_4(rho):
    epsilon = 1e-10
    rho += epsilon * np.eye(rho.shape[0])
    U, s, _ = np.linalg.svd(rho)
    s = np.maximum(s, np.finfo(float).eps) + 1e-10 # evitar erros numéricos
    s = np.nan_to_num(s) # substituir NaN por zero
    return -np.sum(s*np.log2(s))

In [ ]:
def von_neumann_sympy(rho):
    eigenvalues = Matrix(rho).eigenvals()
    nonzero_eigenvalues = [v for v in eigenvalues if v != 0]
    return -sum([p*log(p) for p in nonzero_eigenvalues])

#### Sobre a função 'von_neumann_num_2'

A função apresentada utiliza a decomposição espectral para calcular os autovalores da matriz densidade e, em seguida, calcula o número de Shannon desses autovalores. No entanto, essa abordagem pode apresentar problemas em situações em que a matriz densidade está próxima de um estado puro, pois os autovalores serão muito próximos de 0 ou 1, o que pode levar a erros numéricos.

Uma alternativa é utilizar a decomposição de valores singulares (SVD) em vez da decomposição espectral. A decomposição SVD é mais estável numericamente em situações em que a matriz é próxima a um estado puro, pois não há necessidade de calcular os autovalores. A função modificada poderia ser escrita da seguinte forma:

-->  von_neumann_num_2

Aqui, a função np.linalg.svd é usada para obter a decomposição de valores singulares da matriz densidade. A função np.maximum é usada para evitar erros numéricos, garantindo que os valores singulares sejam maiores do que o menor número positivo de ponto flutuante representável. Em seguida, o número de von Neumann é calculado usando a fórmula S(rho) = -tr(rho log2(rho)) = -sum(s*log2(s)), onde s são os valores singulares da matriz densidade.

Com essa abordagem, é possível obter uma estimativa mais estável e precisa do número de von Neumann em situações em que a matriz densidade está próxima de um estado puro.

Na linha de código U, s, _ = np.linalg.svd(rho), a função np.linalg.svd retorna três valores: a matriz unitária U, os valores singulares s e a matriz unitária transposta V^T.

A decomposição de valores singulares (SVD) é uma maneira de decompor uma matriz em três matrizes:

css
Copy code
A = U * S * V^T

Onde U e V são matrizes unitárias (ou seja, suas inversas são iguais às suas transpostas) e S é uma matriz diagonal com valores singulares não negativos na diagonal.

Nesse contexto, U é a matriz unitária obtida pela decomposição SVD da matriz densidade rho, s é uma matriz 1D com os valores singulares de rho e _ é a matriz unitária transposta V^T, que não é usada na função von_neumann_num.

Esses valores são atribuídos a U, s e _ respectivamente, usando a técnica de descompactação (ou desestruturação) de tuplas do Python, onde os valores são atribuídos a várias variáveis ​​em uma única linha de código. Isso é feito usando a notação de tuplas, que é indicada por parênteses. Nesse caso, U, s e _ são as três variáveis que recebem os valores retornados pela função np.linalg.svd na mesma ordem.

A linha de código s = np.maximum(s, np.finfo(float).eps) na função von_neumann_num modifica os valores singulares da matriz densidade rho antes de calcular o número de von Neumann.

O objetivo dessa linha de código é evitar que o logaritmo da matriz densidade, que é usado para calcular o número de von Neumann, resulte em valores infinitos ou indefinidos quando um ou mais valores singulares são iguais a zero. Isso ocorre porque o logaritmo de zero é indefinido.

Para evitar esse problema, a linha de código usa a função np.maximum para comparar cada valor singular de rho com um valor mínimo, que é definido como a precisão de máquina para o tipo de dados float (ou seja, o menor número positivo que pode ser representado com precisão em um float). Se um valor singular for menor que esse valor mínimo, ele é substituído pelo valor mínimo. Isso garante que todos os valores singulares de rho sejam maiores ou iguais ao valor mínimo e, portanto, evita o problema de logaritmos indefinidos ou infinitos.

Em resumo, a linha de código s = np.maximum(s, np.finfo(float).eps) é uma precaução necessária para lidar com casos em que a matriz densidade rho é quase pura e tem um ou mais valores singulares muito próximos de zero.

##### Continuei tendo problemas numéricos

Quando a matriz densidade está próxima do estado puro, os autovalores da matriz podem se aproximar de zero e causar erros numéricos ao calcular o logaritmo. Para evitar isso, uma opção é adicionar um pequeno valor de correção aos autovalores antes de calcular o logaritmo. Isso é conhecido como "correção de regularização" ou "regularização de Tikhonov". Uma escolha comum para esse valor é o chamado "epsilon de máquina" da máquina utilizada.

Para aplicar essa correção à função von_neumann_num_2, você pode modificar a linha s = np.maximum(s, np.finfo(float).eps) para s = np.maximum(s, np.finfo(float).eps) + 1e-15, por exemplo. O valor adicionado pode ser ajustado dependendo do seu caso específico.

In [ ]:
def von_neumann(rho): # for symbolics
    d = rho.shape[0]; evals = zeros(d,1); ev = eVals(rho)
    return shannon(ev)

### Linear entropy
\begin{align}
S_{L}(\rho)&=1-Tr(\rho^{2})=1-\sum_{j}(\rho\rho)_{j,j} = 1 - \sum_{j,k}\rho_{j,k}\rho_{k,j}= 1 - \sum_{j,k}\rho_{j,k}\rho_{j,k}^{*} \\
&= 1 - \sum_{j,k}|\rho_{j,k}|^{2}
\end{align}
If $\rho=|\psi\rangle\langle\psi|$ then $\rho^{2}=\rho$ and $S_{L}(|\psi\rangle)=0$. If $\rho=\mathbb{I}_{d}/d$ then $\rho^{2} = \mathbb{I}_{d}/d^{2}$ and $S_{L}(\mathbb{I}_{d}/d) = 1-Tr(\mathbb{I}_{d}/d^{2}) = 1-d/d^{2} = (d-1)/d.$ So $\bar{S}_{L}(\rho) = \frac{d}{d-1}S_{L}(\rho)\in[0,1].$

In [ ]:
def linear_entropy(rho):
    d = rho.shape[0]; Sl=0
    for j in range(0,d):
        for k in range(0,d):
            Sl += abs(rho[j,k])**2
    return 1-Sl

## Coerência

### Relative entropy complementarity
\begin{equation}
C_{re}(\rho_{A})+P_{re}(\rho_{A})+Q_{re}(|\psi\rangle_{AB}) = \log_{2}(d_{A})
\end{equation}
with $\rho_{A}=Tr_{B}(|\psi\rangle_{AB}\langle\psi|)$
#### Relative entropy coherence
\begin{equation}
C_{re}(\rho_{A}) = S(\rho^{A}_{diag})-S(\rho_{A}),
\end{equation}
with $\rho^{A}_{diag}=diag(\rho^{A}_{1,1},\rho^{A}_{2,2},\cdots,\rho^{A}_{d,d})$.

In [4]:
# Coherence relative entropy for different basis
def coherence_re(rho, th, ph): # th,ph defines the reference basis
    d = rho.shape[0]; pv = np.zeros(d)
    # matrix representation of rho in the basis defined by (th,ph)
    rhon = rho_basis_n(th, ph, rho, 'n'); #print(rhon)
    for j in range(0, d):
        pv[j] = rhon[j,j].real
    # we use the same state to compute S_vn because of its basis invariance
    return shannon_num(pv) - von_neumann_num(rho)

In [ ]:
def coh_re(rho):
    d = rho.shape[0]; pv = np.zeros(d)#; print(rho)
    for j in range(0,d):
        pv[j] = rho[j,j].real
    #pv = np.array(pv).astype(np.double); rho = np.array(rho).astype(np.cdouble)
    return shannon_num(pv) - von_neumann_num(rho) # for numerics
    #return shannon(pv) - von_neumann(rhoA) # for symbolics

In [ ]:
def coh_re_s(rho):
    d = rho.shape[0]; pv = np.zeros(d)#; print(rho)
    for j in range(0,d):
        pv[j] = rho[j,j].real
    #pv = np.array(pv).astype(np.double); rho = np.array(rho).astype(np.cdouble)
    #eturn shannon_num(pv) - von_neumann_num(rho) # for numerics
    return shannon(pv) - von_neumann(rhoA) # for symbolics

### $l_{1}$-norm complementarity
\begin{equation}
C_{l_{1}}(\rho_{A})+P_{l_{1}}(\rho_{A})+Q_{l_{1}}(\rho_{A})=d_{A}-1
\end{equation}
with $\rho_{A}=Tr_{B}(|\psi\rangle_{AB}\langle\psi|$)
#### $l_{1}$-norm coherence
\begin{equation}
C_{l_{1}}(\rho_{A}) = \sum_{j\ne k}|\rho^{A}_{j,k}| = 2\sum_{j<k}|\rho^{A}_{j,k}|
\end{equation}

In [44]:
def coh_l1(rho):
    d = rho.shape[0]; C = 0
    for j in range(0,d-1):
        for k in range(j+1,d):
            C += np.abs(rho[j,k])
    return 2*C

### Hilbert-Schmidt coherence
\begin{equation}
C_{hs}(\rho) = \sum_{j\ne k}|\rho_{j,k}|^{2} = 2\sum_{j<k}|\rho_{j,k}|^{2}
\end{equation}

In [ ]:
def coh_hs(rho):
    d = rho.shape[0]; C = 0
    for j in range(0,d-1):
        for k in range(j+1,d):
            C += (abs(rho[j,k]))**2
    return 2*C

## Preditibilidade

In [ ]:
def predict_vn(rho):
    d = rho.shape[0]; P = 0
    for j in range(0,d):
        if rho[j,j] > 10**-15 and rho[j,j] < (1.0-10**-15):
            P += np.absolute(rho[j,j])*math.log2(np.absolute(rho[j,j])) # for numerics
            #P += rho[j,j].real*log(rho[j,j].real,2) # is not working
    return log(d,2) + P

## Relative entropy predictability
\begin{equation}
P_{re}(\rho) = P_{vn}(\rho) = \log_{2}(d) + H(\vec{p}) =\log_{2}(d)+\sum_{j=1}^{d}\rho_{jj}\log_{2}(\rho_{jj})
\end{equation}

In [ ]:
def predictability_vn(rho, th, ph): # th,ph define the reference basis
    d = rho.shape[0]; 
    # matrix rep. of rho in the basis def. by (th,ph)
    rhon = rho_basis_n(th, ph, rho, 'n');# print('rhon = ',rhon)
    pv = np.zeros(d); 
    for j in range(0,d):
        pv[j] = rhon[j,j].real
    return log(d, 2) - shannon_num(pv)

### Hilbert-Schmidt-linear predictability
\begin{equation}
P_{hs}(\rho) = \sum_{j}(\rho_{j,j})^{2}-1/d_{A}
\end{equation}

In [ ]:
def predict_hs(rho):
    d=rho.shape[0]; P = 0
    for j in range(0,d):
        P += abs(rho[j,j])**2
    return P-1/d

### $l_{1}$-norm predictability
\begin{equation}
P_{l_{1}}(\rho_{A}) = d-1-\sum_{j\ne k}\sqrt{\rho^{A}_{j,j}\rho^{A}_{k,k}}= d-1-2\sum_{j<k}\sqrt{\rho^{A}_{j,j}\rho_{k,k}}
\end{equation}

In [1]:
def predict_l1(rho):
    d=rho.shape[0]; P = 0
    for j in range(0,d-1):
        for k in range(j+1,d):
            P += sqrt(rho[j,j]*rho[k,k])
    return d-1-2*P

## Emaranhamento

In [ ]:
def Econcurrence(rho):
    R = rho@tp(Pauli(2),Pauli(2))@np.matrix.conjugate(rho)@tp(Pauli(2),Pauli(2))
    ev = lapak.zheevd(R)
    evm = max(abs(ev[0][0]), abs(ev[0][1]), abs(ev[0][2]), abs(ev[0][3]))
    Ec = 2*math.sqrt(abs(evm)) - math.sqrt(abs(ev[0][0])) - math.sqrt(abs(ev[0][1]))\
    - math.sqrt(abs(ev[0][2])) - math.sqrt(abs(ev[0][3]))
    if Ec < 0.0:
        Ec = 0.0
    return Ec

In [ ]:
# Emaranhamento de formação
def Eof(rho):
    Ec = Econcurrence(rho)
    pv = np.zeros(2); pv[0] = (1+math.sqrt(1-Ec**2))/2; pv[1] = 1 - pv[0]
    return shannon_num(pv)

In [ ]:
def concurrence(rho):
    # Converte a matriz rho em um array NumPy de complexos com precisão de 128 bits.
    rho_array = np.array(rho).astype(np.complex128)
    # Calcula o operador de correlação R com as matrizes de Pauli e a matriz rho.
    R = rho_array @ kron(Pauli(2),Pauli(2)) @ conjugate(rho_array) @ kron(Pauli(2),Pauli(2))
    # Calcula os autovalores do operador de correlação R.
    ev = eigvals(R)
    # Calcula o maior valor absoluto dos autovalores.
    evm = max(abs(ev[0]), abs(ev[1]), abs(ev[2]), abs(ev[3]))
    # Calcula a concorrência a partir dos autovalores e do maior valor absoluto.
    C = 2.0*sqrt(abs(evm)) - sqrt(abs(ev[0])) - sqrt(abs(ev[1])) - sqrt(abs(ev[2])) - sqrt(abs(ev[3]))
    # Caso a concurrence seja negativa, retorna 0.
    if C < 0.0:
        C = 0.0
    # Retorna a concurrence calculada.
    return C

## Correlation

### $l_{1}$-norm quantum correlation
\begin{equation}
Q_{l_{1}}(\rho_{AB}) = \sum_{j\ne k}\left(\sqrt{\rho^{A}_{jj}\rho^{A}_{kk}}-|\rho^{A}_{jk}|\right)=2\sum_{j< k}\left(\sqrt{\rho^{A}_{jj}\rho^{A}_{kk}}-|\rho^{A}_{jk}|\right),
\end{equation}
with $\rho^{A}=Tr_{B}(\rho_{AB})$

In [2]:
def qcorr_l1(rhoA):
    da = rhoA.shape[0]; qc = 0
    for j in range(0,da-1):
        for k in range(j+1,da):
            qc += sqrt(rhoA[j,j]*rhoA[k,k]) - abs(rhoA[j,k])
    return 2*qc

## Hilbert-Schmidt-linear quantum correlation
\begin{equation}
Q_{hs}(\rho_{AB}) = S_{l}(\rho_{A}) = 1-Tr(\rho_{A}^{2}).
\end{equation}

In [3]:
def qcorr_hs(rhoA):
    return linear_entropy(rhoA)

## Relative entropy quantum correlation
\begin{equation}
Q_{re}(|\psi\rangle_{AB}) = S_{vn}(\rho_{A})
\end{equation}

In [ ]:
def qcorr_re(rhoA):
    return von_neumann_num(rhoA) # for numerics